In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd

from tensorflow import keras

In [2]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

'C:\\Users\\lucifer\\.keras\\datasets\\auto-mpg.data'

In [3]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [6]:
dataset = dataset.copy().fillna(method="ffill")

In [7]:
origin = dataset.pop('Origin')

dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
0,18.0,8,307.0,130.0,3504.0,12.0,70,1.0,0.0,0.0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1.0,0.0,0.0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1.0,0.0,0.0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1.0,0.0,0.0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1.0,0.0,0.0


In [8]:
train_dataset = dataset.sample(frac=0.75,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [9]:
train_dataset

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
65,14.0,8,351.0,153.0,4129.0,13.0,72,1.0,0.0,0.0
132,25.0,4,140.0,75.0,2542.0,17.0,74,1.0,0.0,0.0
74,13.0,8,302.0,140.0,4294.0,16.0,72,1.0,0.0,0.0
78,21.0,4,120.0,87.0,2979.0,19.5,72,0.0,1.0,0.0
37,18.0,6,232.0,100.0,3288.0,15.5,71,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
301,34.2,4,105.0,70.0,2200.0,13.2,79,1.0,0.0,0.0
130,26.0,4,122.0,80.0,2451.0,16.5,74,1.0,0.0,0.0
187,17.5,8,305.0,140.0,4215.0,13.0,76,1.0,0.0,0.0
23,26.0,4,121.0,113.0,2234.0,12.5,70,0.0,1.0,0.0


In [10]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
minmax = MinMaxScaler(feature_range=(-10,10))

In [21]:
model = keras.Sequential([
    keras.layers.Dense(100,input_shape=[len(dataset.columns)-1]),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.Dense(1)
])

In [22]:
model.compile(
    loss="mse",
    optimizer=keras.optimizers.RMSprop(0.001),
    metrics=["mae","mse"]
)

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               1000      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 6,101
Trainable params: 6,101
Non-trainable params: 0
_________________________________________________________________


In [24]:
X = minmax.fit_transform(train_dataset)

In [26]:
model.fit(X,train_labels,epochs=10)

Epoch 1/10
298/298 [==============================] - 0s 124us/sample - loss: 69.5041 - mean_absolute_error: 6.1755 - mean_squared_error: 69.5041
Epoch 2/10
298/298 [==============================] - 0s 104us/sample - loss: 22.5019 - mean_absolute_error: 3.2322 - mean_squared_error: 22.5019
Epoch 3/10
298/298 [==============================] - 0s 110us/sample - loss: 13.0368 - mean_absolute_error: 2.5232 - mean_squared_error: 13.0368
Epoch 4/10
298/298 [==============================] - 0s 100us/sample - loss: 11.3302 - mean_absolute_error: 2.3720 - mean_squared_error: 11.3302
Epoch 5/10
298/298 [==============================] - 0s 102us/sample - loss: 10.4186 - mean_absolute_error: 2.3475 - mean_squared_error: 10.4186
Epoch 6/10
298/298 [==============================] - 0s 97us/sample - loss: 10.4733 - mean_absolute_error: 2.2830 - mean_squared_error: 10.4733
Epoch 7/10
298/298 [==============================] - 0s 100us/sample - loss: 10.5025 - mean_absolute_error: 2.3073 - mean_sq

### California Housing 

In [38]:
df = pd.read_csv("./housing.csv")

In [39]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [40]:
df = df.fillna(method="ffill")

In [59]:
feature = df.copy()
label = feature.pop('median_house_value')

In [60]:
ocean_prox = pd.get_dummies(df['ocean_proximity']).drop(columns=['NEAR BAY'])

In [61]:
feature = pd.concat([feature,ocean_prox],axis=1).drop(columns=["ocean_proximity"])

In [64]:
minmax_ = MinMaxScaler(feature_range=(-5,5))

In [65]:
minmax_.fit(feature)

MinMaxScaler(copy=True, feature_range=(-5, 5))

In [67]:
feature_scaled = minmax_.transform(feature)

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
X,x,Y,y = train_test_split(feature_scaled,label.values)

In [93]:
model = keras.Sequential([
    keras.layers.Dense(128,input_shape=[X.shape[1]],activation="relu"),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.RMSprop(0.001),
    loss="mse",
    metrics=['mse','mae']
)

In [94]:
model.fit(X,Y,epochs=20)

Epoch 1/20
15480/15480 [==============================] - 1s 87us/sample - loss: 37701439906.0672 - mean_squared_error: 37701419008.0000 - mean_absolute_error: 154810.9844
Epoch 2/20
15480/15480 [==============================] - 1s 79us/sample - loss: 11338427581.1886 - mean_squared_error: 11338426368.0000 - mean_absolute_error: 82390.3984
Epoch 3/20
15480/15480 [==============================] - 1s 87us/sample - loss: 9737923924.2749 - mean_squared_error: 9737922560.0000 - mean_absolute_error: 75653.4062
Epoch 4/20
15480/15480 [==============================] - 1s 81us/sample - loss: 8384422925.7592 - mean_squared_error: 8384421888.0000 - mean_absolute_error: 69676.4688
Epoch 5/20
15480/15480 [==============================] - 1s 85us/sample - loss: 6995900737.2238 - mean_squared_error: 6995903488.0000 - mean_absolute_error: 62699.1250
Epoch 6/20
15480/15480 [==============================] - 1s 85us/sample - loss: 6007816584.9302 - mean_squared_error: 6007813632.0000 - mean_absolute

In [95]:
model.evaluate(x,y)

5160/5160 [==============================] - 0s 52us/sample - loss: 4656699146.7163 - mean_squared_error: 4656699904.0000 - mean_absolute_error: 50002.7461


[4656699146.716279, 4656700000.0, 50002.746]

In [96]:
from sklearn.metrics import mean_squared_error

In [97]:
np.sqrt(mean_squared_error(y,model.predict(x)))

68240.01143853553